# Backtesting

---

### Import Libraries and Dependencies

In [2]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path

### Read CSV into Pandas DataFrame

In [2]:
# Set the file path
filepath = Path("../Resources/aapl.csv")

# Read the CSV located at the file path into a Pandas DataFrame
aapl_df = pd.read_csv(filepath, parse_dates=True, infer_datetime_format=True)

# Print the DataFrame
aapl_df.head()

date   close    volume    open    high     low
0  9/22/14  101.06  52421660  101.80  102.14  100.58
1  9/23/14  102.64  63255860  100.60  102.94  100.54
2  9/24/14  101.75  59974260  102.16  102.85  101.20
3  9/25/14   97.87  99689300  100.51  100.71   97.72
4  9/26/14  100.75  62276770   98.53  100.75   98.40

### Generate a Dual Moving Average Crossover Trading Signal

In [3]:
# Grab just the `date` and `close` from the dataset
signals_df = aapl_df.loc[:, ['date', 'close']].copy()

# Set the `date` column as the index
signals_df = signals_df.set_index("date", drop=True)

# Set the short window and long windows
short_window = 50
long_window = 100

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df['SMA50'] = signals_df['close'].rolling(window=short_window).mean()
signals_df['SMA100'] = signals_df['close'].rolling(window=long_window).mean()
signals_df['Signal'] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the SMA50 is under the SMA100, and
# where 1 is when the SMA50 is higher (or crosses over) the SMA100
signals_df['Signal'][short_window:] = np.where(
    signals_df['SMA50'][short_window:] > signals_df['SMA100'][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df['Entry/Exit'] = signals_df['Signal'].diff()

# Print the DataFrame
signals_df.tail(10)

close     SMA50     SMA100  Signal  Entry/Exit
date                                                     
9/6/19   213.260  205.2584  199.72930     1.0         0.0
9/9/19   214.170  205.5470  199.87850     1.0         0.0
9/10/19  216.700  205.9226  200.01420     1.0         0.0
9/11/19  223.590  206.3634  200.21150     1.0         0.0
9/12/19  223.085  206.7705  200.39705     1.0         0.0
9/13/19  218.750  207.0573  200.50975     1.0         0.0
9/16/19  219.900  207.3707  200.63715     1.0         0.0
9/17/19  220.700  207.7843  200.79135     1.0         0.0
9/18/19  222.770  208.2149  200.97605     1.0         0.0
9/19/19  220.960  208.5695  201.13955     1.0         0.0

### Plot Entry and Exit Points of Dual Moving Average Crossover Trading Strategy

In [4]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['close'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['close'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA50', 'SMA100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I          :Curve   [date]   (close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .Scatter.Close.I  :Scatter   [date]   (close)
   .Scatter.Close.II :Scatter   [date]   (close)

### Backtest the Trading Strategy

In [5]:
# Set initial capital
initial_capital = float(100000)

# Set the share size
share_size = 500

# Take a 500 share position where the dual moving average crossover is 1 (SMA50 is greater than SMA100)
signals_df['Position'] = share_size * signals_df['Signal']

# Find the points in time where a 500 share position is bought or sold
signals_df['Entry/Exit Position'] = signals_df['Position'].diff()

# Multiply share price by entry/exit positions and get the cumulatively sum
signals_df['Portfolio Holdings'] = signals_df['close'] * signals_df['Entry/Exit Position'].cumsum()

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
signals_df['Portfolio Cash'] = initial_capital - (signals_df['close'] * signals_df['Entry/Exit Position']).cumsum()

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + signals_df['Portfolio Holdings']

# Calculate the portfolio daily returns
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()

# Calculate the cumulative returns
signals_df['Portfolio Cumulative Returns'] = (1 + signals_df['Portfolio Daily Returns']).cumprod() - 1

# Print the DataFrame
signals_df.tail(10)

close     SMA50     SMA100  Signal  Entry/Exit  Position  \
date                                                                  
9/6/19   213.260  205.2584  199.72930     1.0         0.0     500.0   
9/9/19   214.170  205.5470  199.87850     1.0         0.0     500.0   
9/10/19  216.700  205.9226  200.01420     1.0         0.0     500.0   
9/11/19  223.590  206.3634  200.21150     1.0         0.0     500.0   
9/12/19  223.085  206.7705  200.39705     1.0         0.0     500.0   
9/13/19  218.750  207.0573  200.50975     1.0         0.0     500.0   
9/16/19  219.900  207.3707  200.63715     1.0         0.0     500.0   
9/17/19  220.700  207.7843  200.79135     1.0         0.0     500.0   
9/18/19  222.770  208.2149  200.97605     1.0         0.0     500.0   
9/19/19  220.960  208.5695  201.13955     1.0         0.0     500.0   

         Entry/Exit Position  Portfolio Holdings  Portfolio Cash  \
date                                                               
9/6/19                   0.0            106630.0         22495.0   
9/9/19                   0.0            107085.0         22495.0   
9/10/19                  0.0            108350.0         22495.0   
9/11/19                  0.0            111795.0         22495.0   
9/12/19                  0.0            111542.5         22495.0   
9/13/19                  0.0            109375.0         22495.0   
9/16/19                  0.0            109950.0         22495.0   
9/17/19                  0.0            110350.0         22495.0   
9/18/19                  0.0            111385.0         22495.0   
9/19/19                  0.0            110480.0         22495.0   

         Portfolio Total  Portfolio Daily Returns  \
date                                                
9/6/19          129125.0                -0.000077   
9/9/19          129580.0                 0.003524   
9/10/19         130845.0                 0.009762   
9/11/19         134290.0                 0.026329   
9/12/19         134037.5                -0.001880   
9/13/19         131870.0                -0.016171   
9/16/19         132445.0                 0.004360   
9/17/19         132845.0                 0.003020   
9/18/19         133880.0                 0.007791   
9/19/19         132975.0                -0.006760   

         Portfolio Cumulative Returns  
date                                   
9/6/19                       0.291250  
9/9/19                       0.295800  
9/10/19                      0.308450  
9/11/19                      0.342900  
9/12/19                      0.340375  
9/13/19                      0.318700  
9/16/19                      0.324450  
9/17/19                      0.328450  
9/18/19                      0.338800  
9/19/19                      0.329750

### Plot Entry/Exit Points of Backtest Results

In [6]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize total portoflio value for the investment
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Overlay plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I                    :Curve   [date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [date]   (Portfolio Total)